# Types of reservoir operating policies
In this Notebook we will see how we can define different operating policies in terms of their shape and variability across the year so the policy can adapt to different operating purposes or variable conditions. The first type of operating policy was already presented in the previous Notebook "3.c. Reservoir operating policy". The operating policy was a piece-wise linear function of the storage that did not change across the year. Here we will see:
- Log-Exponetial function
- Rule curve: a variable (across the year) function

<left><img src="../../util/images/Dam4.gif" width = "500px"><left>
    
Once again we consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir rulve curve that optimizes the **long-term** (several years) system performance. We use the historical time series of inflows and water demand to estimate such long-term performance. The underpinning assumption here is that the system forcings observed over the past years are representative of the forcings that will drive the system in the future (if this assumption is not sensible, for instance because of ongoing changes that will likely impact the hydrological regime or demand pattern, then one may use model projections of inflow and demand in place of historical observations) 
<left> <img src="../../util/images/system_representation_IO1.png" width = "600px"><left>
    
### Import libraries
To run this notebook we need to import some libraries:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from platypus import NSGAII, Problem, Real, Integer # Import the optimizer
# iRONS submodules and functions
from Modules.Interactive_release_policy import Interactive_logexp_manual_v1
from Modules.Interactive_rule_curves import Interactive_rule_curve,Interactive_rule_curve_manual,Interactive_rule_curve_auto
from irons.Functions.Data_management.Read_data import read_csv_data
from irons.Functions.Data_management.day2week2month import day2week
from irons.Functions.Reservoir_operating_policy.Operating_policy import op_piecewiselin_1res, op_logexp_1res_v1
from irons.Functions.Reservoir_operating_policy.Operating_rule_curves import rule_curve

### Loading historical inflows, evaporation and water demand data
Let's assume we want to look at 100 weeks from 2014 to 2015, and load the evaporation, inflow and demand observations for this period from a file.

In [2]:
### Load evaporation data ###
inputs_folder_path = 'Inputs/'
clim_data_file = 'clim_data_2014_15.csv'
date_day, clim_data_day  = read_csv_data(inputs_folder_path, clim_data_file)
date, e_data_week, e_data_week_cum = day2week(date_day,clim_data_day[:,0])
### Load inflow data ###
I_data_file = 'inflow_data_2014_15.csv'
date_day, I_data_day = read_csv_data(inputs_folder_path, I_data_file)
date, I_data_week, I_data_week_cum = day2week(date_day,I_data_day[:,0])
### Load demand data ###
d_data_file = 'demand_data_2014_15.csv'
date_day, d_data_day = read_csv_data(inputs_folder_path, d_data_file)
date, d_data_week, d_data_week_cum = day2week(date_day,d_data_day)
N = len(date) # weeks

### Definition of reservoir simulation function and system characterisitics
Next, we need to import the iRONs function that implements the reservoir simulation (this function iteratively applies the mass balance equation so to reconstruct the temporal evolution of the reservoir variables over the simulation period). We also need to specify the system characteristics.

In [3]:
### Import the reservoir simulation function ###
from irons.Functions.Reservoir_system_simulation.Res_sys_sim import Res_sys_sim

# System characteristics
d             = 15 # ML/week - water demand (we assume as constant) 
env_min       = 4 # ML/week - environmental compensation flow
Qreg_rel_mean = d # ML/week - the (long-term) mean release = demand  

# System constraints
Qreg_rel_min = env_min # ML/week - the release at minimum storage 
Qreg_rel_max = 40 # ML/week - the maximum release capacity 

s_min   = 0 # ML - minimum storage (set to zero for now)
s_max   = 150 #  ML - maximum storage (=reservoir capacity) 

### Define initial storage for simulation ###
s_ini = 100 # ML - initial storage volume

### Definition of the system objectives
Last, we need to define the objectives that we want to be maximised by the rule curve. As anticipated in the Introduction of this Notebook, our illustrative reservoir is operated to support domestic supply while maintaining the reservoir level above a prescribed target (this could be, for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment).
We will pursue the first objective by minimising the following Total Squared Deficit (TDC) with respect to the historical water demand:

$$TSD = \sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-u(t) \ ) \ ]^2 $$

where N is the length of the simulation period and d(t) is the water demand for each time-interval in that period. Notice that the function $max(0,...)$ enables us to only count the difference between demand d and release u when this is positive, that is, when the release u is smaller than the demand d, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. This translates the fact that small deficit amounts are easier to mitigate and hence more acceptable, while larger ones can cause disproportionately severe impacts and should be avoided as much as possible.

We are also interested in minimising the chances that the reservoir level goes below a minimum threshold. We measure how well this criterion is satisfied by the following Critical Storage Violation (CSV) function:

$$CSV = \sum_{t=1}^{N} [ \ max ( \ cs - s(t) , \ 0 \ ) \ ] $$

where, again, N is the length of the simulation period, s is the reservoir storage, and cs is the minimum reservoir storage threshold that should preferably not be transpassed. For our case, let's set this threshold to 40 ML.

In [4]:
cs = np.zeros((N+1,1))+40 # (ML)  minimum reservoir storage threshold

## Log-Exponetial function
The Log-Exponetial function determines the reservoir releases (*Q_reg_rel*) as a function of the storage fraction (*s_frac*). *s_frac* is the reservoir storage scaled by the reservoir active capacity, so that in the operating policy function *s_frac* varies between 0 (dead storage)  and 1 (full storage). This function consists of two segments:
1. reservoir storage below the reference storage (s_frac < s_frac_ref) - Logarithmic behavior:
            
$$Q_{reg,rel} = \ (u_{frac,min} + ln \ ( \ k \ s_{frac}^{α} + 1 \ ) \ ) \ u_{ref}$$
            
2. reservoir storage above the reference storage (s_frac ≥ s_frac_ref) - Exponential behavior:
            
$$Q_{reg,rel} \ = \ exp( \ b \ ( \ s_{frac} \ - \ s_{frac,ref} \ )^2 \ ) \ u_{ref}$$

    where:
$$ k = 1 \ / \ (s_{frac,ref}^{α}) \ (exp \ (1 \ - \ u_{frac,min}) \  - \ 1)$$

*u_frac_min* is the minimum allowed reservoir release scaled by the release at the reference storage, *s_frac_ref* is the reference reservoir storage fraction (when the reservoir is at this level the reservoir release is equal to the reference release e.g. target demand and it varies from 0 to 1), *α* is the shape parameter for the logarithmic segment, *b* is the shape parameter for the exponential segment and *u_ref* is the reference or target release (reservoir release when the *s = s_ref* and it can be defined for instance as the target demand or as the average annual discharge over the past 5 years)

In [5]:
### Operating policy defining points ###
u_ref = Qreg_rel_mean # ML/week - the target demand 
u_frac_min = Qreg_rel_min/u_ref # min release fraction
s_frac_ref = 0.8 # storage fraction at the reference point (transition between the log and the exp segments)

### Operating policy shape parameters ###
α = 1
b = 40

param_1 = [u_frac_min, s_frac_ref, α, b, u_ref]
u = op_logexp_1res_v1(param_1)

We can now create and plot the operating policy with the following code. Play with the sliders to change the values of *s_frac_ref*, *α* and *b* and try to answer to the following questions: 
- If we increase the value of *s_frac_ref* do we increase or decrease the risk of flooding? why?
- By decreasing the value of which parameter we make the reservoir operating policy more prone to release at lower storage? why?

In [6]:
d = d + np.zeros((N,1))
fig_1a,fig_1b,fig_1c, u_frac_min, s_frac_ref, α, b = Interactive_logexp_manual_v1(
                                                                        Res_sys_sim, op_logexp_1res_v1,
                                                                        date,
                                                                        I_data_week, e_data_week, 
                                                                        s_ini, s_min, s_max, 
                                                                        param_1, Qreg_rel_min, Qreg_rel_max, 
                                                                        cs, d)

Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [widgets.HBox([widgets.VBox([s_frac_ref, α, b])],layout=Box_layout), fig_1a],layout=Box_layout),
              fig_1b,fig_1c],layout=Box_layout)

## Rule curve: a variable (across the year) function
A rule curve is essentially an operating policy function that varies across the year. For a rule curve the year is divided into discrete intra-year time periods, for instance days, each period having a specified reservoir operating policy, i.e a function that returns the release volume based on the conditions of the reservoir system (for instance, the reservoir storage, the demand forecast, the time of year, etc.) at that time. In this way, each reservoir operating policy adapts to different conditions across the year, e.g. to seasonal changes of reservoir inflows and demans.

We will use a variable piece-wise linear function where the reservoir release is a only determined by the storage value, as in the Figure below. At normal operation, i.e. at normal storage values, the average historical release is released. Higher storage values are associated to higher releases, which is useful for flood control purposes, whereas at critically low storage values less water is released to reduce the risk of future water shortages [(Loucks et al., 1981)](https://link.springer.com/book/10.1007/978-3-319-44234-1).
<left><img src="../../util/images/Rule_curve.png" width = "800px"><left>

### Defining the rule curve
In our code, storage values are rescaled by the reservoir active capacity, so it varies between 0 (dead storage) and 1 (full storage).
The variable piece-wise linear function is defined by 3 dates: 1 Apr, 1 Aug and 1 Dec. For the rest of the dates the operating policy will be defined by linear interpolation. The operating policy returns a constant release (the average historical release) when the storages stay in the Normal Storage band (between s1 and s2); the release is reduced if the storage is in the Critical Storage band (below s1), or increased if it is in the Flood Storage band (above s2). Let's now attribute a (tentative) value of s1 and s2 in the 3 points, so that we can visualise the variable piece-wise linear function across the year. For example:

In [7]:
### Rule curve parameters ###
# Defining points 
# (linear interpolation will be applied to define the operating policies between the points)
curve_dates = ['1 Apr',    '1 Aug',    '1 Dec',    '31 Mar'] # for 1 Apr and 31 Mar we apply the same value
s_0         = [s_min/s_max,s_min/s_max,s_min/s_max,s_min/s_max]
s_1         = [0.6,        0.3,        0.2,        0.6] 
s_2         = [0.9,        0.6,        0.5,        0.9] 
s_3         = [s_max/s_max,s_max/s_max,s_max/s_max,s_max/s_max]
# Points defining the release policy across the year 
# (linear interpolation will be applied to define the curves between the points)
rule_dates  = ['21 Mar',     '21 Jun',     '21 Sep',     '21 Dec']
u_0         = [Qreg_rel_min, Qreg_rel_min, Qreg_rel_min, Qreg_rel_min]
u_1         = [Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean]
u_2         = [Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean,Qreg_rel_mean]
u_3         = [Qreg_rel_max, Qreg_rel_max, Qreg_rel_max, Qreg_rel_max]

param_2 = {'curves':{'year_date'    : curve_dates,
                   'storage_frac' : [s_0,s_1,s_2,s_3]},
         'rules' :{'year_date' : rule_dates,
                   'release'   : [u_0,u_1,u_2,u_3]}} # max release
s_yday,r_yday = rule_curve(param_2)

We can now plot an interactive figure of the rule curve together with the plots of the operating policity functions in 1 Apr, 1 Aug and 1 Dec with the following code. Use the sliders to modify the parameters of the rule curve at those 3 dates and try to minimise the Total Squared Deficit (TSD) and the Critical Storage Violation (CSV).

In [8]:
d = d_data_week # d + np.zeros(N)
fig_2a,fig_2b,fig_2c,fig_2d,fig_2e,fig_2f,s_1_0_slider,s_1_1_slider,s_1_2_slider = Interactive_rule_curve_manual(
                                                                           Res_sys_sim,op_piecewiselin_1res,rule_curve,
                                                                           date,
                                                                           I_data_week,e_data_week, 
                                                                           s_ini, s_min, s_max, 
                                                                           Qreg_rel_mean, Qreg_rel_min, Qreg_rel_max,
                                                                           cs, d)

Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [widgets.HBox([s_1_0_slider,s_1_1_slider,s_1_2_slider],layout=Box_layout), fig_2a],layout=Box_layout),
              widgets.HBox([fig_2d,fig_2e,fig_2f],layout=Box_layout),
              fig_2b,fig_2c],layout=Box_layout)

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

Loucks D. P. et al (1981) Water resource systems planning and analysis, Prentice-Hall.